# Loading Neils' Inversions

## Imports

In [5]:
import os
import gc
import sys
import pickle

import pandas as pd
import geopandas as gpd
import xarray as xr
import numpy as np

from shapely.geometry import Point

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.pyplot import imshow

import scipy.interpolate as interpolate

## Loading the .dat file

In [6]:
# setup the file location variables
data_dir = '\\\\prod.lan\\active\\proj\\futurex\\StuartCorridor\\Data\\Processed\\Geophysics\\AEM\\NBC_inversion\\line_data_cor2DLog\\'
data_file = 'NBC_Stuart_ModelExp_cor2DLog.dat'
hdr_file = 'NBC_Stuart_ModelExp_cor2DLog.hdr'
working_dir = '\\\\prod.lan\\active\\proj\\futurex\\StuartCorridor\\Working\\Mike\\'

# build the column name list
# first load the header file
cols = list(pd.read_csv(data_dir + hdr_file, delim_whitespace = True, header = None)[1].values)

# now build the column names for each layer
# number of layers
n = 30
# the names of each column to be repeated n times
col_types = cols[-3::]
# drop the repeated column names
cols = cols[0:-3]
xtra_cols = []
# make the new column names
for col_type in col_types:
    for i in range(1,n+1):
        xtra_cols.append(str(i) + '_' + col_type)
# add the new ones to the existing ones
cols = cols + xtra_cols
cols

['sequence',
 'line',
 'easting',
 'northing',
 'elevation',
 'DataResidual',
 'Residual2',
 'Residual3',
 'TotalResidual',
 'TxHeightMeasured',
 'TxHeightInverted',
 'StdDev_TxHeightInverted',
 'DOI',
 'nlayers',
 '1_resistivity',
 '2_resistivity',
 '3_resistivity',
 '4_resistivity',
 '5_resistivity',
 '6_resistivity',
 '7_resistivity',
 '8_resistivity',
 '9_resistivity',
 '10_resistivity',
 '11_resistivity',
 '12_resistivity',
 '13_resistivity',
 '14_resistivity',
 '15_resistivity',
 '16_resistivity',
 '17_resistivity',
 '18_resistivity',
 '19_resistivity',
 '20_resistivity',
 '21_resistivity',
 '22_resistivity',
 '23_resistivity',
 '24_resistivity',
 '25_resistivity',
 '26_resistivity',
 '27_resistivity',
 '28_resistivity',
 '29_resistivity',
 '30_resistivity',
 '1_resistivity_uncertainty',
 '2_resistivity_uncertainty',
 '3_resistivity_uncertainty',
 '4_resistivity_uncertainty',
 '5_resistivity_uncertainty',
 '6_resistivity_uncertainty',
 '7_resistivity_uncertainty',
 '8_resistivity

In [7]:
# load the data, using the column names created above
data = pd.read_csv(data_dir + data_file, delim_whitespace = True, header = None, names = cols)
data['geometry'] = data.apply(lambda x: Point((float(x.easting), float(x.northing))), axis=1)
data = gpd.GeoDataFrame(data, geometry = 'geometry')
data.crs = {'init': 'espg:28353'}
data

,sequence,line,easting,northing,elevation,DataResidual,Residual2,Residual3,TotalResidual,TxHeightMeasured,...,22_depth_top,23_depth_top,24_depth_top,25_depth_top,26_depth_top,27_depth_top,28_depth_top,29_depth_top,30_depth_top,geometry
0,1,100101,424827.7,7369579.0,466.83,1.321,0.0,0.035,0.138,32.67,...,151.870,176.310,204.660,237.550,275.690,319.950,371.300,430.87,499.99,POINT (424827.7 7369579)
1,2,100101,424811.1,7369578.0,466.77,1.292,0.0,0.033,0.137,32.31,...,151.870,176.310,204.660,237.550,275.690,319.950,371.300,430.87,499.99,POINT (424811.1 7369578)
2,3,100101,424794.2,7369577.0,466.79,1.226,0.0,0.032,0.133,31.96,...,151.870,176.310,204.660,237.550,275.690,319.950,371.300,430.87,499.99,POINT (424794.2 7369577)
3,4,100101,424777.5,7369575.0,466.89,1.191,0.0,0.027,0.131,31.64,...,151.870,176.310,204.660,237.550,275.690,319.950,371.300,430.87,499.99,POINT (424777.5 7369575)
4,5,100101,424761.0,7369574.0,467.00,1.173,0.0,0.026,0.130,31.38,...,151.870,176.310,204.660,237.550,275.690,319.950,371.300,430.87,499.99,POINT (424761 7369574)
5,6,100101,424744.5,7369573.0,467.05,1.202,0.0,0.028,0.131,31.18,...,151.870,176.310,204.660,237.550,275.690,319.950,371.300,430.87,499.99,POINT (424744.5 7369573)
6,7,100101,424728.0,7369573.0,467.02,1.341,0.0,0.035,0.139,31.02,...,151.870,176.310,204.660,237.550,275.690,319.950,371.300,430.87,499.99,POINT (424728 7369573)
7,8,100101,424711.5,7369572.0,466.95,1.607,0.0,0.042,0.153,30.88,...,151.870,176.310,204.660,237.550,275.690,319.950,371.300,430.87,499.99,POINT (424711.5 7369572)
8,9,100101,424694.8,7369571.0,466.90,1.786,0.0,0.052,0.162,30.73,...,151.870,176.310,204.660,237.550,275.690,319.950,371.300,430.87,499.99,POINT (424694.8 7369571)
9,10,100101,424677.9,7369571.0,466.91,2.001,0.0,0.059,0.172,30.56,...,151.870,176.310,204.660,237.550,275.690,319.950,371.300,430.87,499.99,POINT (424677.9 7369571)


## Choosing Ti Tree Study Area

In [8]:
# cut the data just to the Ti Tree study area
east_min = 288000
east_max = 426000
north_min = 7450000
north_max = 7600000
data = data[(data['easting'] >= east_min) & (data['easting'] <= east_max) & (data['northing'] >= north_min) & (data['northing'] <= north_max)]

data

,sequence,line,easting,northing,elevation,DataResidual,Residual2,Residual3,TotalResidual,TxHeightMeasured,...,22_depth_top,23_depth_top,24_depth_top,25_depth_top,26_depth_top,27_depth_top,28_depth_top,29_depth_top,30_depth_top,geometry
152088,13017,118101,337788.3,7512893.0,599.70,20.552,0.0,0.000,0.538,32.46,...,151.870,176.310,204.660,237.550,275.690,319.950,371.300,430.87,499.99,POINT (337788.3 7512893)
152089,13018,118101,337803.9,7512889.0,599.62,20.590,0.0,0.000,0.539,32.46,...,151.870,176.310,204.660,237.550,275.690,319.950,371.300,430.87,499.99,POINT (337803.9 7512889)
152090,13019,118101,337819.6,7512885.0,599.58,20.646,0.0,0.000,0.539,32.46,...,151.870,176.310,204.660,237.550,275.690,319.950,371.300,430.87,499.99,POINT (337819.6 7512885)
152091,13020,118101,337835.2,7512882.0,599.56,20.715,0.0,0.000,0.540,32.48,...,151.870,176.310,204.660,237.550,275.690,319.950,371.300,430.87,499.99,POINT (337835.2 7512882)
152092,13021,118101,337850.6,7512878.0,599.56,20.781,0.0,0.000,0.541,32.51,...,151.870,176.310,204.660,237.550,275.690,319.950,371.300,430.87,499.99,POINT (337850.6 7512878)
152093,13022,118101,337866.0,7512874.0,599.57,20.833,0.0,0.000,0.542,32.54,...,151.870,176.310,204.660,237.550,275.690,319.950,371.300,430.87,499.99,POINT (337866 7512874)
152094,13023,118101,337881.7,7512871.0,599.56,20.870,0.0,0.000,0.542,32.58,...,151.870,176.310,204.660,237.550,275.690,319.950,371.300,430.87,499.99,POINT (337881.7 7512871)
152095,13024,118101,337897.5,7512867.0,599.53,20.890,0.0,0.000,0.542,32.64,...,151.870,176.310,204.660,237.550,275.690,319.950,371.300,430.87,499.99,POINT (337897.5 7512867)
152096,13025,118101,337913.3,7512864.0,599.47,20.894,0.0,0.000,0.542,32.71,...,151.870,176.310,204.660,237.550,275.690,319.950,371.300,430.87,499.99,POINT (337913.3 7512864)
152097,13026,118101,337929.0,7512861.0,599.37,20.888,0.0,0.000,0.542,32.81,...,151.870,176.310,204.660,237.550,275.690,319.950,371.300,430.87,499.99,POINT (337929 7512861)


## Export Shapefile

In [7]:
data.to_file(filename = "NeilssPoints.shp", driver='ESRI Shapefile')

## Gridding the Data

http://gdal.org/python/osgeo.gdal-module.html#Grid

In [ ]:
# define the pixel size (in metres) in the resulting grids
pixel_size = 400

# define the range for the meshgrid required for the interpolation
x_axis = np.arange(east_min, east_max + pixel_size, pixel_size)
y_axis = np.arange(north_min, north_max + pixel_size)
# make the meshgrid
x, y = np.meshgrid(x_axis, y_axis)

# define the list of resistivity columns
resistivity_cols = [str(i) + '_resistivity' for i in range(1,31)]

l = len(resistivity_cols)
llen = len(str(l))

# do the interpolation for a single grid at a time through all 30 layers of the AEM inversion
# save these out to conserve memory
for i, res_col in enumerate(resistivity_cols):
    ans = interpolate.griddata(data[['easting','northing']], data[res_col], (x,y) , method = 'cubic')
    with open(working_dir + res_col + '.pkl', 'wb') as f:
        pickle.dump(ans, f)
    
    # progress tracker
    pc = (i + 1)/ l
    barlen = 50
    progress = chr(9608) * int(pc * barlen) + ' ' * int(barlen* (1 - pc))
    string = ' Progress:' + progress + '| ' + str(int(pc * 100)) + ' % done'
    sys.stdout.flush()
    nstr = (llen - len(str(i))) * ' ' + str(i)
    print('Number: ' + nstr + string, end='\r')

In [ ]:
# freeing up some memory
# del data
# gc.collect()

# make a list of all the pickle files
pkls = [file for file in os.listdir(working_dir) if file[-4::] == '.pkl']
grids = []
# loop through and load all pickle files
for pkl in pkls:
    with open(working_dir + pkl, 'rb') as f:
        grids.append(pickle.load(f))
grids

In [ ]:
# freeing up some memory
# del data
# gc.collect()

# make a list of all the pickle files
pkls = [file for file in os.listdir(working_dir) if file[-4::] == '.pkl']

# loop through and load all pickle files
for pkl in pkls:
    with open(working_dir + pkl, 'rb') as f:
        imshow(pickle.load(f).T)
        

In [ ]:
pkls = [file for file in os.listdir(working_dir) if file[-4::] == '.pkl']
pkl = pkls[0]
with open(working_dir + pkl, 'rb') as f:
    imshow(pickle.load(f).T)
        

In [ ]:
# stack the grids along a new axis
grids = np.stack(grids)

# use the exisitng axes as the axes label for the impending Xarray
x_axis = x_axis.astype(np.float32)
y_axis = y_axis.astype(np.float32)

# make the z dimesion labels, including dealing with rounding errors for the top depth for each layer.
# for TiTree, the largest discrepancy was 1.5cm.
depth_cols = [str(i) + '_depth_top' for i in range(1,n+1)]
z = np.around([data[col].mode()[0] for col in depth_cols],2).astype(np.float32)

# assemble the Xarray
da = xr.DataArray(grids, coords=[('z', z), ('x',x_axis),('y',y_axis)], dims =['z','x','y']).astype(np.float32)

del grids
gc.collect()
da

In [ ]:
# save the output as a netCDF for future use
